## Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests

# ----------- Science Jubilee -------------
from science_jubilee import Machine as Jub
from science_jubilee.tools import HTTPSyringe as syringe
from science_jubilee.tools import Pipette, PneumaticSampleLoader
from science_jubilee.utils import Handlers
import time
import numpy as np
import pandas as pd
import logging
import sys
sys.path.append('../..')
import stober_synthesis_utils as stober

In [ ]:
FORMAT = '%(asctime)s:%(levelname)s:%(name)s:%(message)s'
logging.basicConfig(filename = '2025_03_18_mesoporoustest.log', level = logging.INFO, format = FORMAT)
logger = logging.getLogger(__name__)

In [ ]:
jubilee = Jub.Machine(address='192.168.1.2', simulated = False, crash_detection = False) 

In [ ]:
sample_table = pd.read_csv('2025_03_11_mesoporous_sampletable - Sheet1.csv')
#sample_table_2 = pd.read_csv('SampleTable_ReproReplicates_10000.csv')


In [ ]:
deck = jubilee.load_deck('lab_automation_deck_AFL_bolton.json')

In [ ]:
sample_table['well'] = ['A1', 'A2', 'A3', 'A4', 'B1']

In [ ]:
sample_table

In [ ]:
jubilee.park_tool()

In [ ]:
sample_table['water_volume'].sum()

## Load Tools

In [ ]:
syringe_10 = syringe.HTTPSyringe.from_config(2, "../../../science-jubilee/src/science_jubilee/tools/configs/10cc_syringe.json")
syringe_1_1 = syringe.HTTPSyringe.from_config(1, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_1_syringe.json")
syringe_1_2 = syringe.HTTPSyringe.from_config(3, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_2_Hamiltonsyringe.json")
syringe_1_3 = syringe.HTTPSyringe.from_config(4, "../../../science-jubilee/src/science_jubilee/tools/configs/1cc_3_Hamiltonsyringe.json")
syringe_1_4 = syringe.HTTPSyringe.from_config(0, '../../../science-jubilee/src/science_jubilee/tools/configs/1cc_4_syringe.json')

In [ ]:
jubilee.load_tool(syringe_10)
jubilee.load_tool(syringe_1_1)
jubilee.load_tool(syringe_1_2)
jubilee.load_tool(syringe_1_3)
jubilee.load_tool(syringe_1_4)

In [ ]:
mix_syringe = syringe_10
water_syringe = syringe_1_1
ammonia_syringe = syringe_1_2
teos_syringe = syringe_1_3
surfactant_syringe = syringe_1_4

## Load water syringe with water

In [ ]:
water_syringe.load_syringe(600, 1500)

In [ ]:
water_syringe.set_pulsewidth(water_syringe.empty_position-1, s = 2000)

In [ ]:
water_syringe.set_pulsewidth(water_syringe.full_position+1, s = 150)

In [ ]:
water_syringe.set_pulsewidth(water_syringe.full_position+250, s = 500)

In [ ]:
water_syringe.load_syringe(590, water_syringe.full_position+250)

## Load ammonia syringe

In [ ]:
#ammonia_syringe.set_pulsewidth(1420, s = 10)

In [ ]:
ammonia_syringe.load_syringe(600, 1500)

In [ ]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.empty_position-1, s = 2000)

In [ ]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+1, s = 200)

In [ ]:
ammonia_syringe.set_pulsewidth(ammonia_syringe.full_position+250, s = 500)

In [ ]:
ammonia_syringe.load_syringe(560, ammonia_syringe.full_position+250)

## Load TEOS syringe

In [ ]:
teos_syringe.load_syringe(600, 1500)

In [ ]:
teos_syringe.set_pulsewidth(teos_syringe.empty_position-1, s = 2000)

In [ ]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+1, s = 200)

In [ ]:
teos_syringe.set_pulsewidth(teos_syringe.full_position+300, s = 200)

In [ ]:
teos_syringe.load_syringe(510, teos_syringe.full_position+300)

## load ethanol syringe

In [ ]:
#jubilee.park_tool()

In [ ]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

In [ ]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1, s = 2000)


In [ ]:
#mix_syringe.empty_position

In [ ]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

## Don't need to load surfactant syringe

In [ ]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)

## Load AFL

In [ ]:
safe_pos = (5, 200, 100)
cell_pos = (217.5, 69.0, 61.5)
url = 'http://192.168.1.4'
port = '5000'
name = 'PSL'
username = 'test'
password = 'domo_arigato'

In [ ]:
psl = PneumaticSampleLoader.PneumaticSampleLoader(url, port, name, cell_pos, safe_pos, username, password)

In [ ]:
jubilee.load_tool(psl)

## Load labware


In [ ]:
samples = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 2)
samples.manual_offset([(31.1, 169.0), (121.1, 169.0), (121.1, 113.0)])

In [ ]:
stocks_main = jubilee.load_labware('20mlscintillation_12_wellplate_18000ul.json', 4)
stocks_main.manual_offset([(30.5, 266.0),  (120.5, 266.0), (120.5, 210.0)])

In [ ]:
teos_stocks = [stocks_main[0]]
ammonia_stocks = [stocks_main[1]]
water_stocks = [stocks_main[2]]
ethanol_stocks = [stocks_main[3]]
ctab_stocks = [stocks_main[4]]
f127_stocks = [stocks_main[5]]
rinse_stocks_precursor = [stocks_main[6], stocks_main[7], stocks_main[8]]
rinse_stocks_teos = [stocks_main[9], stocks_main[10], stocks_main[11]]

## Check sample alignment

In [ ]:
jubilee.pickup_tool(mix_syringe)
for i, row in sample_table.iterrows():
    loc = samples[row['well']]
    jubilee.move_to(x = loc.x, y = loc.y)
    print(loc.y)
    jubilee.move_to(z = loc.top_+7)
    curr_well = row['well']
    ans = input(f'Is the tip centered over sample {curr_well}?')
    if ans == 'y':
        continue
    else:
        continue
    

In [ ]:
sample_table

In [ ]:
jubilee.park_tool()


In [ ]:
loc = samples['A8']
jubilee.move_to(x = loc.x, y = loc.y)
print(loc.y)
jubilee.move_to(z = loc.top_+7)

# Experiment


In [ ]:
location_lookup = {row['uuid']:samples[row['well']] for i, row in sample_table.iterrows()}


In [ ]:
sample_table['ethanol_volume'].sum()

In [ ]:
sample_table_select = sample_table.iloc[1:2]

In [ ]:
sample_table_select

In [ ]:
psl.rinse_cell()

## Load ethanol for background

In [ ]:
psl.load_sample(mix_syringe, ethanol_stocks[0], 1000)

## add ethanol

In [ ]:
def reactant_transfer(syringe, source, destination, volume, volume_buffer, rinse_stocks = None, rinse_vol = 500, n_rinse = 1, dwell_time = 3):
    # zero out syringe
    syringe.set_pulsewidth(syringe.empty_position - 1)
    syringe.load_syringe(0, syringe.empty_position-1)

    jubilee.pickup_tool(syringe)

    
    # need to account for dispenses > syringe volume
    n_dispenses = int(np.ceil(volume / (syringe.capacity - volume_buffer)))
    step_volume = volume/n_dispenses
    print(f'breaking dispense into {n_dispenses} of volume {step_volume}')
    
    for i in range(n_dispenses):
        # make sure syringe has enough volume
        if i == 0:
            syringe.aspirate(step_volume + volume_buffer, source.bottom(+5), s = 10, dwell_before = 5)
        else:
            syringe.aspirate(step_volume, source.bottom(+5), s = 10, dwell_before = 5)

    
            

        syringe.dispense(step_volume, destination.bottom(+5), s = 200)
        time.sleep(dwell_time)
        logger.info(f'Dispensed {volume} uL from {source} into {destination}')

    syringe.dispense(volume_buffer, source.bottom(+5), s = 20)

    if rinse_stocks is not None:
        for stock in rinse_stocks:
            syringe.mix(rinse_vol, n_rinse, stock.bottom(+10), t_hold = 3, s_aspirate = 2000, s_dispense = 1000)

        logger.info(f'Washed mix syringe in wash solutions {rinse_stocks}')

    jubilee.park_tool()
    syringe.set_pulsewidth(syringe.empty_position - 1)
    syringe.load_syringe(0, syringe.empty_position-1)


In [ ]:
ind = 5

well = location_lookup[ind]
etoh_vol = sample_table.iloc[ind-1]['ethanol_volume']
f127_volume = sample_table.iloc[ind-1]['f127_voume']
ctab_volume = sample_table.iloc[ind-1]['ctab_volume']
sample_table_select = sample_table.iloc[ind-1:ind]

reaction_time = 60*5

In [ ]:
f127_volume

In [ ]:
well

In [ ]:

start_time = time.time()

#add ethanol
reactant_transfer(mix_syringe, ethanol_stocks[0], well, etoh_vol, 500)

#add water
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

#add ammonia
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup, 'ammonia_volume', ammonia_stocks, dwell_time = 10)

# add F127

reactant_transfer(surfactant_syringe, f127_stocks[0], well, f127_volume, 50)

reactant_transfer(surfactant_syringe, ctab_stocks[0], well, ctab_volume, 50)

# mix precursors

stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, rinse_stocks_precursor, 3, n_rinse = 1)

# add teos
teos_add_time = stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1, return_time = True)

print(f'Time from TEOS addition to load: {time.time() - teos_add_time} s')

time.sleep(reaction_time - (time.time() - teos_add_time))




In [ ]:
psl.load_sample(mix_syringe, well, 1000)



mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

print(f'Time to load: {(time.time() - start_time)/60}')
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock, s_aspirate = 2000, s_dispense = 1000)

jubilee.park_tool()

end_time = time.time()

print(f'Total sample time: {(end_time - start_time)/60}')

In [ ]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock)

In [ ]:
sample_table

In [ ]:
etoh_vol

In [ ]:
time.time()

In [ ]:
time.time() + 60*5 

In [ ]:
stober.add_reactants_batch(jubilee, mix_syringe, mix_syringe, sample_table_select, location_lookup, 'ethanol_volume', ethanol_stocks, stocks_usable_volume=15000, dwell_time = 10)

In [ ]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)

In [ ]:
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)

## Dispense water


In [ ]:
stober.add_reactants_batch(jubilee, water_syringe, mix_syringe, sample_table_select, location_lookup, 'water_volume', water_stocks, dwell_time = 7)

## dispense Ammonia

In [ ]:
stober.add_reactants_batch(jubilee, ammonia_syringe, mix_syringe, sample_table_select, location_lookup, 'ammonia_volume', ammonia_stocks, dwell_time = 10)

## dispense F127

In [ ]:
stober.add_reactants_batch(jubilee, surfactant_syringe, mix_syringe, sample_table_select, location_lookup, 'f127_voume', f127_stocks, refill_dwell = 10)

In [ ]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)
jubilee.pickup_tool(surfactant_syringe)
for stock in rinse_stocks_precursor:
    surfactant_syringe.mix(900, 1, stock)
jubilee.park_tool()

## Dispense CTAB

In [ ]:
stober.add_reactants_batch(jubilee, surfactant_syringe, mix_syringe, sample_table_select, location_lookup, 'ctab_volume', ctab_stocks, refill_dwell = 10)

In [ ]:
surfactant_syringe.set_pulsewidth(surfactant_syringe.empty_position-1)
surfactant_syringe.load_syringe(0, surfactant_syringe.empty_position-1)
jubilee.pickup_tool(surfactant_syringe)
for stock in rinse_stocks_precursor:
    surfactant_syringe.mix(900, 1, stock)
jubilee.park_tool()

## mix precursors

In [ ]:
ind = 2
location_lookup_selected = {ind:location_lookup[ind]}

In [ ]:
location_lookup_selected

In [ ]:
stober.first_mix(jubilee, mix_syringe, 5000, location_lookup_selected, rinse_stocks_precursor, 3, n_rinse = 1)

## Add TEOS and mix

In [ ]:
stober.add_reactants_batch(jubilee, teos_syringe, mix_syringe, sample_table_select, location_lookup, 'teos_volume', teos_stocks, mix_after=(8000, 5, rinse_stocks_teos), wait = False, n_rinse = 1)

## Load cell

In [ ]:
psl.load_sample(mix_syringe, samples['A2'], 1000)

In [ ]:
mix_syringe.set_pulsewidth(mix_syringe.empty_position-1)
mix_syringe.load_syringe(0, mix_syringe.empty_position-1)
jubilee.pickup_tool(mix_syringe)
for stock in rinse_stocks_teos:
    mix_syringe.mix(5000, 3, stock)
jubilee.park_tool()